In [31]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import json
import csv
import pandas
import re
from pathlib import Path
from datetime import datetime
from pandas.io.json import json_normalize 
import matplotlib.pyplot as plt 

In [51]:
now = datetime.now()
s3_prefix = now.strftime("%Y_%m_%d_%H_%M_%S")
print("s3_prefix =", s3_prefix)

s3_prefix = 2020_01_28_12_19_18


In [52]:
# Now run populate-queue to take the unique list of urls and send them to be parsed by the worker
## TODO: run command to tu

In [53]:
%%bash -s "$s3_prefix"
export AWS_PROFILE="bmccarthy" && node src/populate-queue.js -i output/unique_url.txt -p $1 -c 2


{ verbose: false,
  input: 'output/unique_url.txt',
  count: '2',
  prefix: '2020_01_28_12_19_18' }
DONE!
Finished sending 2 messages to https://sqs.us-east-1.amazonaws.com/109628666462/html-feature-extractor-worker-queue. Ending now..


In [54]:
%%bash -s "$s3_prefix"
aws s3 cp s3://html-feature-extractor-worker-output/$1/ output/$1 --recursive --exclude "*" --include "*.json" --profile bmccarthy

In [24]:
pathlist = Path('output/' + s3_prefix).glob('**/*.json')
for path in pathlist:
    path_in_str = str(path)
    with open(path_in_str) as f: 
        d = json.load(f) 
        df = json_normalize(d["elements"]) 
        df.append()



In [17]:
with open('output/losangeles.eventful.com!events!categories!music.json') as f: 
    d = json.load(f) 
    df = json_normalize(d["elements"]) 

    df.fontSize = df.fontSize.apply(lambda x : re.sub("[^0-9]", "", x))
    df.lineHeight = df.lineHeight.apply(lambda x : re.sub("[^0-9]", "", x))
    df.itemType = df.itemType.apply(lambda x : "http://schema.org/Event" == x)

    y = df.itemType
    X = df.drop(labels=["color", "elementId", "tagName", "classList", "textContent", "font", "fontStyle", "backgroundColor", "background", "backgroundImage", "itemType", "itemProp"], axis=1)

    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

    print("Whole set:", X.shape, y.shape)
    print("Training set:", xTrain.shape, yTrain.shape)
    print("Testing set:", xTest.shape, yTest.shape)



Whole set: (1411, 8) (1411,)
Training set: (1128, 8) (1128,)
Testing set: (283, 8) (283,)
UsageError: Line magic function `%%bash` not found.
